In [12]:


# Create a list of the tickers
import os
import requests
import json
import pandas as pd
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
import streamlit as st
from api_caller import api_call
from helper_clean import historical_data_cleaner 
import numpy as np

In [13]:
# Read in the csv file
# Read in the csv file
testing_csv = pd.read_csv('project_1_mockdata.csv')

# Create a list of the tickers
tickers = testing_csv['Ticker'].to_list()

alpaca = api_call()

timeframe = "1Day"

# Create a start and end variables to pull data during this period
end = pd.Timestamp("2020-07-13", tz="America/New_York").isoformat()
start = pd.Timestamp("2020-07-05", tz="America/New_York").isoformat()

# Pull the stock data for our tickers and selected times
df_portfolio = alpaca.get_bars(
    tickers,
    timeframe,
    start = start,
    end = end
).df

def make_daily_close(pulled_df, tickers):
    
    # Create an empty list to hold the dictionaries for each day
    closing_data = []
    
    # Iterate through each unique timestamp to grab each stocks data
    for date in pulled_df.index.unique():
        
        # Create a dictionary to hold the timestamp, and each stocks closing price
        day_data = {}
        
        # Store the current timestamp that we are pulling data for
        day_data['Timestamp'] = date
        
        # Iterate through the tickers for that day, and grab each ones stock price
        for stock in tickers:
            day_data[stock] = pulled_df.loc[(pulled_df.index == date) & (pulled_df['symbol'] == stock), 'close'].item()
        
        # Append the current timestamps data to the list
        closing_data.append(day_data)
        
    # Create a dataframe from the list of closing price dictionaries
    cleaned_data = pd.DataFrame(closing_data)
    
    # Return the dataframe
    return cleaned_data

# Call the function to return the cleaned daily data for our portfolio
cleaned_df = make_daily_close(df_portfolio, tickers)
cleaned_df.head()



,Timestamp,AMZN,GOOG,WMT,JPM
0,2020-07-06 04:00:00+00:00,3057.04,1495.70,118.89,95.00
1,2020-07-07 04:00:00+00:00,3000.12,1485.18,126.95,92.32
2,2020-07-08 04:00:00+00:00,3081.11,1496.00,124.44,93.30
3,2020-07-09 04:00:00+00:00,3182.63,1510.99,127.75,91.28
4,2020-07-10 04:00:00+00:00,3200.00,1541.74,130.68,96.27


In [18]:
x = cleaned_df.iloc[-1,1:].to_list()

x

[3104.0, 1511.34, 129.52, 97.65]

In [45]:
y = testing_csv["Quantity"].to_list()

y

[5, 7, 20, 25]

In [16]:
total = np.dot(x, y)

In [17]:
total

31131.03

In [50]:
ticker_weights = {}
for ticker in tickers:
    stock_price = cleaned_df[ticker].to_list()[-1]
    stock_quantity = testing_csv.loc[testing_csv['Ticker'] == ticker, 'Quantity'].item()
    stock_position = stock_price * stock_quantity
    ticker_weights[ticker] = stock_position / total

    

In [51]:
ticker_weights

{'AMZN': 0.49853795393213785,
 'GOOG': 0.33983392133186724,
 'WMT': 0.08320958220784858,
 'JPM': 0.07841854252814635}